In [ ]:
!pip install requests_html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.8.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 10.4 which is incom

In [ ]:
!pip install yahoo_fin

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=3f008e4f3afd1495a5e702061b8a57ddfab04c4a3ff902c25bd7dc5f51376ff6
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import numpy as np
import gymnasium as gym
import gymnasium.spaces
import random
from stable_baselines3 import SAC
from stable_baselines3.common.env_checker import check_env
from yahoo_fin import stock_info as si
import time

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [ ]:
class StockTradingEnv(gym.Env):
    def __init__(self, stock_symbol, lookback=10, initial_balance=10000):
        super().__init__()
        self.stock_symbol = stock_symbol
        self.lookback = lookback
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.shares_held = 0
        self.net_worth = initial_balance
        self.previous_net_worth = initial_balance
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(lookback,), dtype=np.float32)
        self.price_history = []

    def _get_live_price(self):
        try:
            stock = yf.Ticker(self.stock_symbol)
            price = stock.history(period='1m').iloc[-1]['Close']
            return price if not np.isnan(price) else self.price_history[-1]
        except:
            return self.price_history[-1] if self.price_history else 100.0

    def _get_observation(self):
        price = self._get_live_price()
        self.price_history.append(price)
        if len(self.price_history) > self.lookback:
            self.price_history.pop(0)
        return np.array(self.price_history[-self.lookback:], dtype=np.float32).reshape(-1)

    def step(self, action):
        current_price = self._get_live_price()
        num_shares = int(action[0] * (self.balance / current_price))

        if num_shares > 0:
            cost = num_shares * current_price
            if self.balance >= cost:
                self.shares_held += num_shares
                self.balance -= cost
        elif num_shares < 0:
            num_shares = abs(num_shares)
            if self.shares_held >= num_shares:
                self.shares_held -= num_shares
                self.balance += num_shares * current_price

        self.previous_net_worth = self.net_worth
        self.net_worth = self.balance + (self.shares_held * current_price)
        reward = self.net_worth - self.previous_net_worth
        done = False
        return self._get_observation(), reward, done, False, {}

    def reset(self, seed=None, options=None):
        self.balance = self.initial_balance
        self.shares_held = 0
        self.net_worth = self.initial_balance
        self.previous_net_worth = self.initial_balance
        self.price_history = [self._get_live_price()] * self.lookback
        return np.array(self.price_history, dtype=np.float32).reshape(-1), {}

def train_rl_agent(env, timesteps=10000):
    model = SAC("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=timesteps)
    return model


In [ ]:
if __name__ == "__main__":
    stock_symbol = "ICICIBANK"
    env = StockTradingEnv(stock_symbol)
    check_env(env)
    model = train_rl_agent(env, timesteps=10000)

    obs, _ = env.reset()
    for _ in range(10):
        action, _states = model.predict(obs)
        obs, reward, done, _, _ = env.step(action)
        print(f"Action: {action[0]:.2f}, Balance: {env.balance:.2f}, Shares Held: {env.shares_held}, Reward: {reward:.2f}")
        time.sleep(1)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Action: -0.53, Balance: 10000.00, Shares Held: 0, Reward: 0.00
Action: 0.55, Balance: 4600.00, Shares Held: 54, Reward: 0.00
Action: 0.43, Balance: 2700.00, Shares Held: 73, Reward: 0.00
Action: -0.86, Balance: 5000.00, Shares Held: 50, Reward: 0.00
Action: 0.21, Balance: 4000.00, Shares Held: 60, Reward: 0.00
Action: -0.97, Balance: 7800.00, Shares Held: 22, Reward: 0.00
Action: -0.11, Balance: 8600.00, Shares Held: 14, Reward: 0.00
Action: -0.50, Balance: 8600.00, Shares Held: 14, Reward: 0.00
Action: -0.20, Balance: 8600.00, Shares Held: 14, Reward: 0.00
Action: -0.83, Balance: 8600.00, Shares Held: 14, Reward: 0.00
